In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [3]:
# 비디오 파일 이름
input_file = "D:\\Pitch\\sample_videos\\14_left.mp4" # 여기에 비디오 경로 넣으면 됩니다!

# 비디오 파일의 경우 이것을 사용하세요:
cap = cv2.VideoCapture(input_file)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [4]:
# 출력 파일 경로 및 이름 설정
output_file = os.path.splitext(input_file)[0] + "_dotted.mp4"
out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

In [8]:
# 어깨 관절 좌표화 코드
right_shoulder_coordinates = [] # 우완투수, 위 리스트와 둘 중 하나는 주석처리해야 합니다.

with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("비디오 파일을 처리하는데 실패했습니다.")
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        image_height, image_width, _ = image.shape

        if results.pose_landmarks:

            right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            cv2.circle(image, (int(right_shoulder.x * image_width), int(right_shoulder.y * image_height)), 5, (0, 0, 255), -1)
            right_shoulder_coordinates.append((right_shoulder.x, right_shoulder.y))

        out.write(image)

        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
out.release()  # 출력 파일 닫기
cv2.destroyAllWindows()

비디오 파일을 처리하는데 실패했습니다.


In [9]:
# 팔꿈치 좌표 출력
for i, coord in enumerate(right_shoulder_coordinates): # 왼손투수의 경우 left_shoulder_coordinates로 바꿔주세요.
    print(f"Frame {i+1}: x = {coord[0] * image_width}, y = {coord[1] * image_height}")

Frame 1: x = 832.4824690818787, y = 884.9365425109863
Frame 2: x = 832.469208240509, y = 884.8141479492188
Frame 3: x = 832.6350331306458, y = 884.707088470459
Frame 4: x = 832.8340744972229, y = 884.2268943786621
Frame 5: x = 832.7176880836487, y = 883.8118171691895
Frame 6: x = 832.6679921150208, y = 883.5275459289551
Frame 7: x = 832.6690220832825, y = 883.2446479797363
Frame 8: x = 832.61958360672, y = 883.099193572998
Frame 9: x = 832.6397323608398, y = 883.0983352661133
Frame 10: x = 832.8208136558533, y = 883.0987358093262
Frame 11: x = 832.9111933708191, y = 883.1308364868164
Frame 12: x = 833.030412197113, y = 883.2318878173828
Frame 13: x = 833.1227874755859, y = 883.3137130737305
Frame 14: x = 833.1605100631714, y = 883.3024406433105
Frame 15: x = 833.3272361755371, y = 883.3003807067871
Frame 16: x = 833.3310341835022, y = 883.30078125
Frame 17: x = 833.3811163902283, y = 883.300952911377
Frame 18: x = 833.4940910339355, y = 883.2835006713867
Frame 19: x = 833.546941280365,

In [10]:
# 좌표를 데이터프레임에 저장
import pandas as pd
df = pd.DataFrame(right_shoulder_coordinates, columns=["x", "y"]) # x_back, x_left, x_right, y_back, y_left, y_right 중 선택
df.head()

,x,y
0,0.770817,0.460904
1,0.770805,0.460841
2,0.770958,0.460785
3,0.771143,0.460535
4,0.771035,0.460319


In [11]:
output_dir = "D:/Pitch/coordinates"
output_filename = os.path.basename(os.path.splitext(input_file)[0]) + "_shoulder_coordinates.csv"
output_file_path = os.path.join(output_dir, output_filename)

df.to_csv(output_file_path, index=False)

In [12]:
import cv2

cap = cv2.VideoCapture(output_file)

if (cap.isOpened()== False): 
    print("비디오를 열 수 없습니다.")
  
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        cv2.imshow('Frame', frame)
        # q 키를 눌러 비디오 재생을 중지합니다.
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else: 
        break

cap.release()
cv2.destroyAllWindows()

# q 누르면 비디오 중지됩니다!!! 제발!!!